In [12]:
import tensorflow as tf
import keras_tuner
import numpy as np


import pandas as pd
import openpyxl # Needed for reading excel
import pathlib

import decomposition
import models
import data
from metrics import smape
from windower import WindowGenerator
import hp_training
import results

In [13]:
cwd = pathlib.Path.cwd()

code_directory = cwd.parents[1]
gonem_directory = code_directory / "notebooks" / "Gonem"
model_directory = gonem_directory / 'models'

model = 'SS'
product = 'maize'
model_path =  model_directory / f"{model}_{product}"
checkpoint_path = model_path / f"{model}_{product}"


In [14]:
df = data.get_data(directory_path=gonem_directory, product=product)
df = df.iloc[:-2]

In [15]:
label_columns = ['price']
label_columns = df.columns[df.columns.get_level_values(0).isin(label_columns)].tolist()
label_columns

[('price', 'Brazil'),
 ('price', 'France'),
 ('price', 'Germany'),
 ('price', 'Global'),
 ('price', 'Hungary'),
 ('price', 'Ukraine')]

In [16]:
stl = decomposition.STLDecomposer(labels=label_columns, period=12)
log = decomposition.Logger(labels=label_columns)
std = decomposition.Standardizer()

preproc = decomposition.Processor().add(stl).add(log).add(std)

In [17]:
width = 24
label_width = 6
shift = 6

window = WindowGenerator(input_width=width, label_width=label_width, shift=shift, data=df, 
                    # train_begin=0, train_end=.9, val_begin=None, val_end=.96,
                    train_begin=0, train_end=.97, val_begin=None, val_end=None,
                    # train_begin=0, train_end=.5, val_begin=None, val_end=.8,
                    test_begin=None, test_end=1., connect=True, remove_labels=True, label_columns=label_columns)
window.preprocess(preproc)
window

Total window size: 30
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
Label indices: [24 25 26 27 28 29]
Label column name(s): [('price', 'Brazil'), ('price', 'France'), ('price', 'Germany'), ('price', 'Global'), ('price', 'Hungary'), ('price', 'Ukraine')]

In [21]:
label_std = decomposition.Standardizer(mean=std.mean[window.label_columns], std=std.std[window.label_columns])
label_log = decomposition.Logger(label_indices=range(len(window.label_columns)))
postproc = decomposition.Processor().add(label_std).add(label_log)
window.add_label_postprocess(postproc)

In [22]:
inputs = tf.convert_to_tensor(np.load(model_path /  f"{product}_inputs.npy"))
labels = tf.convert_to_tensor(np.load(model_path /  f"{product}_labels.npy"))
weights = tf.convert_to_tensor(np.load(model_path / f"{product}_weights.npy"))
predictions = tf.convert_to_tensor(np.load(model_path / f"{product}_predictions.npy"))
mcds = tf.convert_to_tensor(np.load(model_path / f"{product}_mcd_predictions.npy"))

In [23]:
results.weight_results(weights[0], window.train_df.columns)

,country,price_trend,price_seasonal,price_residual,AVG_TAVG,Corn Price Futures,MAX_TMAX,MIN_TMIN,OBS_VALUE_QUANTITY_TON,OIL_PRICE $/bbl,agricultural_land,energy_use_kg_of_oil_equivalent_per_capita,fossil_fuel_energy_consumption_perc_of_total,gdp_current_us,population_total,renewable_energy_consumption_perc_of_total,unemployment_total,price
0,Brazil,0.392538,9.824404e-01,0.999798,0.999947,NaN,9.849353e-01,0.500000,0.528138,NaN,0.294489,0.500000,0.002461,0.055568,5.000000e-01,0.499983,0.500000,NaN
1,France,1.000000,1.766272e-10,1.000000,0.216082,NaN,5.000130e-01,0.542473,0.500000,NaN,0.000537,0.000108,0.004323,0.499991,3.430957e-07,0.456903,0.992554,NaN
2,Germany,0.438094,9.999728e-01,0.500061,0.005002,NaN,5.654812e-08,0.508717,0.209303,NaN,0.586533,0.500084,0.000016,0.487185,4.998086e-01,0.000617,0.500517,NaN
3,Global,0.998352,9.620469e-01,0.577049,NaN,0.499208,NaN,NaN,NaN,0.000087,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Hungary,0.993907,9.998809e-01,0.270154,0.367875,NaN,9.975214e-01,0.500003,0.075880,NaN,0.999868,0.500008,0.000003,0.452855,1.182533e-08,0.079961,0.000263,NaN
5,Ukraine,0.999954,8.915747e-01,0.485885,0.499999,NaN,1.852136e-02,0.499998,0.500001,NaN,0.991953,0.500000,0.499970,0.999142,5.001532e-01,0.497066,0.073851,NaN


In [24]:
intervals = results.forecast_interval(mcds, 0.1)

In [25]:
import metrics
for i in range(6):
    print(metrics.smape(predictions[:, 5, i], labels[:, 5, i])/2)

tf.Tensor(15.3990345, shape=(), dtype=float32)
tf.Tensor(9.173777, shape=(), dtype=float32)
tf.Tensor(23.208067, shape=(), dtype=float32)
tf.Tensor(9.676351, shape=(), dtype=float32)
tf.Tensor(8.310388, shape=(), dtype=float32)
tf.Tensor(9.810068, shape=(), dtype=float32)
